In [1]:
import os
os.environ["PYTHON_EXEC"] = "/Users/jessieyu/.pyenv/versions/qka-demo/bin/python3"
os.environ["NTC_DOC_FILE"] = "runtime/qka_doc.json"
os.environ["NTC_PROGRAM_FILE"] = "runtime/qka_runtime.py"

In [2]:
from qiskit import IBMQ

In [3]:
provider = IBMQ.load_account()

In [4]:
provider.runtime.programs()

Runtime Program QKA:
  Description: Quantum kernel alignment algorithm.
  Parameters:
    feature_map:
      description: quantum feature map
      type: FeatureMapQuantumControl
    data:
      description: NxD array, where N is the number of data points, D is the feature dimension.
      type: numpy.ndarray
    labels:
      description: Nx1 array of +/-1, where N is the number of data points
      type: numpy.ndarray
    lambda_plus:
      description: Plus kernel parameter
      type: numpy.ndarray
    lambda_minus:
      description: Minus kernel parameter
      type: numpy.ndarray
    C:
      description: penalty parameter for soft-margin
      type: float
  Returns:
    cost_plus:
      description: SVM objective function evaluated at (alpha_+, lambda_+)
      type: float
    cost_minus:
      description: SVM objective function evaluated at (alpha_-, lambda_-)
      type: float


In [5]:
from qka.qka_utils import SPSA_parameters, spsa_step_one, spsa_step_two
from qka.kernel_matrix import KernelMatrix
from qiskit.aqua import QuantumInstance

In [6]:
def align_kernel(backend, feature_map, data, labels, lambda_initial=None, spsa_steps=10, C=1):
    """Align the quantum kernel.

    Uses SPSA for minimization wrt kernel parameters (lambda) and
    gradient ascent for maximization wrt support vector weights (alpha):

    min max cost_function

    Args:
        data (numpy.ndarray): NxD array, where N is the number of data points, D is the feature dimension.
        labels (numpy.ndarray): Nx1 array of +/-1, where N is the number of data points
        lambda_initial (array): initial lambdas for feature map
        spsa_steps (int): number of SPSA steps
        C (float): penalty parameter for soft-margin

    Returns:
        lambdas (array): list of best kernel parameters averaged over the last 10 SPSA steps
        cost_plus_save (list): SVM objective function evaluated at (alpha_+, lambda_+) at each SPSA step
        cost_minus_save (list): SVM objective function evaluated at (alpha_-, lambda_-) at each SPSA step
        cost_final_save (list): estimate of updated SVM objective function F using average of F(alpha_+, lambda_+) and F(alpha_-, lambda_-) at each SPSA step
        lambda_save(list): kernel parameters updated at each SPSA step
        kernel_best (array): the final kernel matrix evaluated with best set of parameters averaged over the last 10 SPSA steps.
        program_data: the full Results object for the final kernel matrix of each SPSA iteration

    """

    lambdas = lambda_initial
    kernel_matrix = KernelMatrix(feature_map, QuantumInstance(backend))

    # Pre-computed spsa parameters:
    spsa_params = SPSA_parameters()

    # Save data at each SPSA run in the following lists:
    lambda_save = []       # updated kernel parameters after each spsa step

    # #####################
    # Start the alignment:

    for count in range(spsa_steps):

        print('SPSA step {} of {}:'.format(count+1, spsa_steps))

        lambda_plus, lambda_minus, delta = spsa_step_one(lambdas=lambdas, spsa_params=spsa_params, count=count)
        
        # Call QKA runtime program.
        runtime_params = {
            'feature_map': feature_map.to_dict(),
            'data': data,
            'labels': labels,
            'lambda_plus': lambda_plus,
            'lambda_minus': lambda_minus,
            'C': C
        }
        provider = backend.provider()
        cost_plus, cost_minus = provider.runtime.run(program_name="QKA",
                                                     backend=backend,
                                                     params=runtime_params).result()
        
        cost_final, lambda_best = spsa_step_two(cost_plus=cost_plus, cost_minus=cost_minus,
                                                lambdas=lambdas, spsa_params=spsa_params, delta=delta, count=count)

        print('\033[92m Cost: {}\033[00m'.format(cost_final))

        lambdas = lambda_best  # updated kernel parameters

        lambda_save.append(lambdas)

    # Evaluate aligned kernel matrix with best set of parameters averaged over last 10 steps:
    lambdas = np.sum(np.array(lambda_save)[-10:, :],axis = 0)/10
    kernel_best = kernel_matrix.construct_kernel_matrix(x1_vec=data, x2_vec=data, parameters=lambdas)

    return {'best_kernel_parameters': lambdas,
            'best_kernel_matrix': kernel_best}


In [7]:
import numpy as np
from qka.featuremaps import FeatureMapQuantumControl

In [8]:
num_samples=5  # number of samples per class in the input data
num_features=2 # number of features in the input data
depth=2        # depth of the feature map circuit
C=1            # SVM soft-margin penalty
spsa_steps=2  # number of SPSA iterations

backend = provider.backend.ibmq_qasm_simulator

# create random test data and labels:
x_train = np.random.rand(2*num_samples, num_features)
y_train = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

# Define the feature map and its initial parameters:
fm = FeatureMapQuantumControl(feature_dimension=num_features, depth=depth, entangler_map=None)
lambda_initial = np.random.uniform(-1,1, size=(fm._num_parameters))

# Align the quantum kernel:
qka_results = align_kernel(backend=backend, feature_map=fm,
                           data=x_train, labels=y_train,
                           lambda_initial=lambda_initial,
                           spsa_steps=spsa_steps, C=C)
print(f"best_kernel_parameters: {qka_results['best_kernel_parameters']}")
print(f"best_kernel_matrix: {qka_results['best_kernel_matrix']}")

SPSA step 1 of 2:
 Cost: 6.493570204085309
SPSA step 2 of 2:
 Cost: 6.074246109672153


/Users/jessieyu/.pyenv/versions/3.8.1/envs/qka-demo/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqbackend.py:800: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  return super().run(circuits, job_name=job_name, job_share_level=job_share_level,


best_kernel_parameters: [ 0.03331271  0.16630372  0.18078404 -0.0822165 ]
best_kernel_matrix: [[1.         0.15917969 0.46972656 0.20117188 0.19824219 0.01953125
  0.57421875 0.26464844 0.82617188 0.59472656]
 [0.15917969 1.         0.26660156 0.71679688 0.203125   0.07128906
  0.08007812 0.21191406 0.38769531 0.109375  ]
 [0.46972656 0.26660156 1.         0.14746094 0.65429688 0.26269531
  0.17285156 0.04003906 0.39355469 0.26367188]
 [0.20117188 0.71679688 0.14746094 1.         0.0859375  0.21191406
  0.29980469 0.65039062 0.17578125 0.26171875]
 [0.19824219 0.203125   0.65429688 0.0859375  1.         0.59375
  0.03417969 0.1953125  0.16699219 0.14160156]
 [0.01953125 0.07128906 0.26269531 0.21191406 0.59375    1.
  0.27148438 0.60644531 0.01757812 0.28808594]
 [0.57421875 0.08007812 0.17285156 0.29980469 0.03417969 0.27148438
  1.         0.34960938 0.25097656 0.95800781]
 [0.26464844 0.21191406 0.04003906 0.65039062 0.1953125  0.60644531
  0.34960938 1.         0.20996094 0.2841796